This notebook could be directly excuted in google colab. The location should fit to the path variable.
This github project was our main reference in developing this pipeline https://github.com/intellygenta/KDDCup2021/blob/main/20210531/code.py
We took time to understand the ensembling approach and integrate our own metrics

In [ ]:
!pip install stumpy>=1.5
!pip install stumpy

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
from google.colab import drive
import pathlib
filename ='drive/MyDrive/phase_1/070_UCR_Anomaly_17555.txt'
drive.mount('/content/drive/')
path = pathlib.Path('/content/drive/MyDrive/phase_2')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import pathlib
import tqdm
import stumpy


In [ ]:
txt_dirpath = path  # Place the txt files in this directory

# Parameter setting
min_window_size = 40
max_window_size = 800
growth_rate = 1.1
denom_threshold = 0.1
upper_threshold = 0.75
lower_threshold = 0.25
const_threshold = 0.05
min_coef = 0.5
small_quantile = 0.1
padding_length = 3
train_length = 10
use_gpu = True

In [ ]:
# Determine window sizes
size = int(np.log(max_window_size / min_window_size) / np.log(growth_rate)) + 1
rates = np.full(size, growth_rate) ** np.arange(size)
ws = (min_window_size * rates).astype(int)

# Select stump function
if use_gpu:
    stump = stumpy.gpu_stump
else:
    stump = stumpy.stump

# Anomaly score names
names = ['zscore',
         'tukey1',
         'tukey2',
    'orig_p2p',
    'diff_p2p',
    'acc_p2p',
    'orig_p2p_inv',
    'diff_small',
    'acc_std',
    'acc_std_inv'
]

In [ ]:
txt_dirpath

PosixPath('/content/drive/MyDrive/phase_2')

In [ ]:
def compute_score(X, number, split, w):
        
    # original data
    seq = pd.DataFrame(X, columns=['orig'])

    # velocity (diff) and acceleration (acc) 
    seq['diff'] = seq['orig'].diff(1)
    seq['acc'] = seq['diff'].diff(1)    

    #Interquartile: TUKEY
    q1 = seq['diff'].rolling(window=w).quantile(lower_threshold)
    q2 = seq['diff'].rolling(window=w).quantile(upper_threshold)
    min = seq['diff'].rolling(window=w).min()
    seq['tukey1'] = ((q1 - min) / (q2 - q1)).shift(-w)

    q1 = seq['acc'].rolling(window=w).quantile(lower_threshold)
    q2 = seq['acc'].rolling(window=w).quantile(upper_threshold)
    min = seq['acc'].rolling(window=w).min()
    seq['tukey2'] = ((q1 - min) / (q2 - q1)).shift(-w)


    #Zscore
    col_mean = seq['orig'].rolling(window=w).mean()
    col_std = seq['orig'].rolling(window=w).std()
    seq["zscore"] = abs((seq["orig"] - col_mean)/col_std).shift(-w)

    # standard deviation (std)
    for name in ['orig', 'acc']:
        seq[f'{name}_std'] = seq[name].rolling(w).std().shift(-w)
    
    #  1st order and 2nd order amplitude
    for name in ['orig', 'diff', 'acc']:
        rolling_max = seq[name].rolling(w).max()
        rolling_min = seq[name].rolling(w).min()
        seq[f'{name}_p2p'] = (rolling_max - rolling_min).shift(-w)
        print(seq)
    
    # Interquartile: diff small
    diff_abs = seq['diff'].abs()
    cond = diff_abs <= diff_abs.quantile(small_quantile)
    seq['diff_small'] = cond.rolling(w).mean().shift(-w)
    
    # inverse (inv)
    for name in ['orig_p2p', 'acc_std']:
        numer = seq[name].mean()
        denom = seq[name].clip(lower=numer * denom_threshold)
        seq[f'{name}_inv'] = numer / denom
    
    # coef for penalizing subsequences with little change
    name = 'orig_p2p'
    mean = seq[name].mean()
    upper = mean * upper_threshold
    lower = mean * lower_threshold
    const = mean * const_threshold
    seq['coef'] = (seq[name] - lower) / (upper - lower)
    seq['coef'].clip(upper=1.0, lower=0.0, inplace=True)
    cond = (seq[name] <= const).rolling(2 * w).max().shift(-w) == 1
    seq.loc[cond, 'coef'] = 0.0
        

    
    # Smooth and mask anomaly score
    padding = w * padding_length
    seq['mask'] = 0.0
    seq.loc[seq.index[w:-w-padding], 'mask'] = 1.0
    seq['mask'] = seq['mask'].rolling(padding, min_periods=1).sum() / padding
    for name in names:
        seq[f'{name}_score'] = seq[name].rolling(w).mean() * seq['mask']
    
    return seq

# Evaluate anomaly score for each time series
results = []
for txt_filepath in sorted(txt_dirpath.iterdir()):
    
    # Load time series
    X = np.loadtxt(txt_filepath)
    number = txt_filepath.stem.split('_')[0]
    split = int(txt_filepath.stem.split('_')[-1])
    #print(f'\n{txt_filepath.name} {split}/{len(X)}', flush=True)
    
    # Evaluate anomaly score for each window size w
    for w in tqdm.tqdm(ws):
        
        # Skip long subsequence
        if w * train_length > split:
            continue
            
        # Compute anomaly score
        seq = compute_score(X, number, split, w)
        
        # Skip if coef is small
        if seq['coef'].mean() < min_coef:
            continue
            
        # Evaluate anomaly score
        for name in names:
            
            # Copy anomaly score
            y = seq[f'{name}_score'].copy()
            print(y)
            
            # Find local maxima
            cond = (y == y.rolling(w, center=True, min_periods=1).max())
            y.loc[~cond] = np.nan
            
            # Find 1st peak
            index1 = y.idxmax()
            value1 = y.max()
            
            # Skip if all score is NaN
            if not np.isfinite(value1):
                continue
                
            # Skip if train data has 1st peak
            begin = index1 - w
            end = index1 + w
            if begin < split:
                continue

            # Find 2nd peak
            y.iloc[begin:end] = np.nan
            index2 = y.idxmax()
            value2 = y.max()
            
            # Skip if 2nd peak height is zero
            if value2 == 0:
                continue
            
            # Evaluate rate of 1st peak height to 2nd peak height
            rate = value1 / value2
            results.append([number, w, name, rate, begin, end, index1, value1, index2, value2])


In [ ]:
# Display results
results = pd.DataFrame(results, columns=['number', 'w', 'name', 'rate', 'begin', 'end', 'index1', 'value1', 'index2', 'value2'])
submission.to_csv('/content/drive/MyDrive/result11.csv')
# Make submission csv
submission = results.loc[results.groupby('number')['rate'].idxmax(), 'index1']
submission.index = np.arange(len(submission)) + 1
submission.name = 'location'
submission.index.name = 'No.'
submission.to_csv('result.csv')

In [ ]:
submission1 = results.loc[results.groupby('number')['rate'].idxmax()]
submission1=submission1[["number","index1","name"]]
submission1['number'] = submission1['number'].astype(int)




In [ ]:
submission1 = submission1.rename(columns={'name': 'anomalymethod'})

In [ ]:
results1 = pd.read_csv('drive/MyDrive/metadata.csv')

In [ ]:
df1=submission1

In [ ]:
df2=results1

In [ ]:
df1

In [ ]:
df1

In [ ]:
df2

In [ ]:
df1 = df1.set_index('number')
df1.index = pd.to_numeric(df1.index, errors='coerce')
df2 = results1.set_index('data_id')
eval = df1.join(df2).reset_index()

In [ ]:
eval

In [ ]:
df3=pd.DataFrame(columns=['name','anomalyfunction'])

In [ ]:
for i in range (len(eval)):
    if eval.loc[i,'anomaly_start'] <= eval.loc[i,'index1'] and eval.loc[i,'index1'] <= eval.loc[i,'anomaly_end']:
        eval.loc[i,'algo_eval'] = True  
        print(eval.loc[i,'name'],eval.loc[i,'anomalymethod'])


    else:
        eval.loc[i,'algo_eval'] = False

In [ ]:
eval

In [ ]:
rslt_df = eval[eval['algo_eval'] == True]

In [ ]:
rslt_df["anomalymethod"].value_counts()

In [ ]:
eval["algo_eval"].value_counts()

True     117
False     83
Name: algo_eval, dtype: int64

In [ ]:
eval.to_csv('./out.csv')  